In [1]:
import torch
import os
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
from tqdm import tqdm

# FGSM Attack: Adding noise directly
def fgsm_attack(image, epsilon):
    noise = epsilon * torch.sign(torch.randn_like(image))  # Random sign perturbation
    adv_image = image + noise
    return torch.clamp(adv_image, 0, 1)  # Keep pixel values valid

# PGD Attack: Iterative noise
def pgd_attack(image, epsilon, alpha, iters):
    adv_image = image.clone().detach()
    for _ in range(iters):
        noise = alpha * torch.sign(torch.randn_like(adv_image))
        adv_image = adv_image + noise
        adv_image = torch.clamp(adv_image, image - epsilon, image + epsilon)  # Projection step
        adv_image = torch.clamp(adv_image, 0, 1)
    return adv_image

In [3]:
# Paths
holdout_path = "datasets/kitti_holdout/image_2"
fgsm_output_path = "datasets/kitti_holdout_FGSM/image_2"
pgd_output_path = "datasets/kitti_holdout_PGD/image_2"

# Ensure output directories exist
os.makedirs(fgsm_output_path, exist_ok=True)
os.makedirs(pgd_output_path, exist_ok=True)

# Transform to tensor
transform = transforms.Compose([transforms.ToTensor()])

# Parameters
epsilon = 0.2  # Perturbation strength
alpha = 0.05  # PGD step size
pgd_iters = 40  # PGD iterations

# Process all images in the holdout dataset
for filename in tqdm(os.listdir(holdout_path), desc="Generating Adversarial Images"):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        image_path = os.path.join(holdout_path, filename)

        # Load image
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Apply FGSM
        adv_fgsm = fgsm_attack(image_tensor, epsilon)
        save_image(adv_fgsm, os.path.join(fgsm_output_path, filename))

        # Apply PGD
        adv_pgd = pgd_attack(image_tensor, epsilon, alpha, pgd_iters)
        save_image(adv_pgd, os.path.join(pgd_output_path, filename))

print("✅ FGSM & PGD adversarial datasets created successfully!")

Generating Adversarial Images: 100%|██████████| 34/34 [00:28<00:00,  1.20it/s]

✅ FGSM & PGD adversarial datasets created successfully!
